[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_arima_diagnostics/EMQA_arima_diagnostics.ipynb)

# EMQA_arima_diagnostics

**ARIMA Residual Diagnostics**

Fit ARIMA(1,1,1) to Brent crude oil prices and run a full residual diagnostic suite: time series, histogram, ACF, and QQ-plot.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import stats

brent = fetch('BZ=F')

# Fit ARIMA(1,1,1)
model = ARIMA(brent, order=(1, 1, 1))
result = model.fit()
print(result.summary())


In [ ]:
resid = result.resid
std_resid = (resid - resid.mean()) / resid.std()

fig, axes = plt.subplots(2, 2, figsize=(14, 9))

# Panel 1: Standardized residuals
axes[0, 0].plot(std_resid.index, std_resid.values, color=COLORS['blue'], linewidth=0.5, alpha=0.8)
axes[0, 0].axhline(0, color=COLORS['gray'], linestyle='--', linewidth=0.5)
axes[0, 0].axhline(2, color=COLORS['red'], linestyle=':', linewidth=0.8, alpha=0.5)
axes[0, 0].axhline(-2, color=COLORS['red'], linestyle=':', linewidth=0.8, alpha=0.5)
axes[0, 0].set_title('Standardized Residuals', fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Std. Residual')

# Panel 2: Histogram + Normal
axes[0, 1].hist(std_resid, bins=50, density=True, color=COLORS['blue'], alpha=0.6, edgecolor='white')
x_range = np.linspace(-4, 4, 200)
axes[0, 1].plot(x_range, stats.norm.pdf(x_range), color=COLORS['red'], linewidth=2, label='Normal')
axes[0, 1].set_title('Residual Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Standardized Residual')
axes[0, 1].set_ylabel('Density')
axes[0, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False)

# Panel 3: ACF of residuals
nlags = 30
acf_vals = acf(resid.dropna(), nlags=nlags)
conf = 1.96 / np.sqrt(len(resid.dropna()))
axes[1, 0].bar(range(nlags + 1), acf_vals, width=0.4, color=COLORS['green'], alpha=0.85)
axes[1, 0].axhline(conf, color=COLORS['red'], linestyle='--', linewidth=0.8)
axes[1, 0].axhline(-conf, color=COLORS['red'], linestyle='--', linewidth=0.8)
axes[1, 0].axhline(0, color='black', linewidth=0.5)
axes[1, 0].set_title('ACF of Residuals', fontweight='bold')
axes[1, 0].set_xlabel('Lag')
axes[1, 0].set_ylabel('ACF')

# Panel 4: QQ-plot
stats.probplot(std_resid.dropna(), dist='norm', plot=axes[1, 1])
axes[1, 1].get_lines()[0].set(color=COLORS['blue'], markersize=3, alpha=0.6)
axes[1, 1].get_lines()[1].set(color=COLORS['red'], linewidth=1.5)
axes[1, 1].set_title('Normal QQ-Plot', fontweight='bold')

fig.suptitle('ARIMA(1,1,1) Residual Diagnostics — Brent Oil', fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
save_fig(fig, 'arima_diagnostics.pdf')
plt.show()


In [ ]:
# Ljung-Box test
lb_test = acorr_ljungbox(resid.dropna(), lags=[10, 20, 30], return_df=True)
print("Ljung-Box Test Results:")
print(lb_test)
print()
if (lb_test['lb_pvalue'] > 0.05).all():
    print("=> No significant autocorrelation in residuals (good model fit)")
else:
    print("=> Some residual autocorrelation detected (consider adjusting the model)")
